In [2]:
from dotenv import load_dotenv
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import Settings
from llama_index.llms.google_genai import GoogleGenAI
import os

Settings.embed_model =  OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
)

Settings.llm = GoogleGenAI(
    model="gemini-2.5-flash",
    api_key=os.getenv("GEMINI_API_KEY"),  
)

load_dotenv()

True

In [3]:
docs = SimpleDirectoryReader(
    input_files=[
        r"C:\Users\ijona\Desktop\DocumentIngestionPipeline-AllCognix\samples\uber_2021.pdf",
        r"C:\Users\ijona\Desktop\DocumentIngestionPipeline-AllCognix\samples\paul_graham_essay.txt",
        r"C:\Users\ijona\Desktop\DocumentIngestionPipeline-AllCognix\samples\RAFT.pdf"
]
).load_data()

In [4]:
print("Total Documents:",len(docs))

Total Documents: 320


In [5]:
print("Document ID:", docs[0].doc_id)

Document ID: b0119f90-78d5-4175-a56b-31a7933b9e6b


In [8]:
collection_name = os.getenv("MILVUS_COLLECTION")

vector_store = MilvusVectorStore(
    uri=os.getenv("MILVUS_ENDPOINT"),
    token=os.getenv("MILVUS_TOKEN"),
    collection_name=collection_name,
    dim=768,
    # overwrite=True, # drop the collection if it already exists
    # enable_sparse=True  # enable the default full-text search using BM25
)

In [10]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    docs, storage_context=storage_context, show_progress=True
)

Parsing nodes:   4%|▍         | 12/320 [00:00<00:02, 107.41it/s]

Generating embeddings: 100%|██████████| 437/437 [00:49<00:00,  8.84it/s]


In [18]:
chat_engine = index.as_chat_engine(
    chat_mode="best"
)

In [28]:
response = await chat_engine.achat(
    "In the art world what are the things that are tightly coupled"
)
print(response.response)


2026-01-12 08:18:46,678 - INFO - AFC is enabled with max remote calls: 10.


2026-01-12 08:18:49,891 - INFO - Condensed question: What are the things that are tightly coupled in the art world?
2026-01-12 08:18:55,431 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2026-01-12 08:18:56,130 - INFO - AFC is enabled with max remote calls: 10.


Based on the documents, here are some things that are described as tightly coupled in the art world:

1.  **Signature Style and Artist Identification/Market Value:** A "signature style" is described as something that "immediately identifies the work as yours and no one else's." This style is tightly coupled with the artist's identity, making their work instantly recognizable, like Roy Lichtenstein's cartoon-like paintings. Furthermore, this signature style is often tightly coupled with **market value**, as buyers "pay a lot for such work" because it's identifiable and unique to a specific artist.

2.  **Painting Still Lives and Curiosity/Close Observation:** The author found a tight coupling between painting still lives and his own **curiosity** about what he was seeing. He explains that painting still lives forces one to "look more closely" and notice details that are usually taken for granted in everyday visual perception. This process of painting still lives is inherently linked to 